In [1]:
import cv2
import os
import math
import numpy as np 
import pandas as pd
from PIL import Image

In [3]:
def video_capturing_function(folder_name):
    dirs = os.listdir('UCF50')
    for d in dirs :
        files = os.listdir(f'UCF50/{d}')
        for file in files:
            video_name=file
            video_read_path=os.path.join(f'UCF50/{d}',video_name)
            cap=cv2.VideoCapture(video_read_path)
            try:          
                os.mkdir(os.path.join(f'UCF50/{d}',
                                      video_name.split(".")[0]))
            except:
                print("File Already Created")

            train_write_file=os.path.join(os.path.join('UCF50',d),
                                      video_name.split(".")[0])
            cap.set(cv2.CAP_PROP_FPS, 1)
            frameRate=cap.get(5)
            x=1
            count=0
            while(cap.isOpened()):
                frameId = cap.get(1) #current frame number
                ret, frame = cap.read()
                if (ret != True):
                    break
                if (frameId % math.floor(frameRate) == 0):
                    filename ="frame%d.jpg" % count;count+=1
                    frame_grey=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    cv2.imwrite(os.path.join(train_write_file,filename), frame_grey)
            cap.release()
    return print("All frames written in the: "+folder_name+" Folder")
    
video_capturing_function("Train_Videos")

All frames written in the: Train_Videos Folder


In [3]:
def data_load_frames(directory):
    frames=[]
    labels = []
    dirs = os.listdir(directory)
    for d in dirs :
        sub_dirs = os.listdir(os.path.join(directory,d))
        for sd in sub_dirs:
            files = os.listdir(os.path.join(directory,d, sd))
            for file in files :
                vid_name=file
                vid_dir_path=os.path.join(directory,d, sd) # to change
                frames_to_select=[]
                for l in np.arange(0,8):
                    frames_to_select.append('frame%d.jpg' % l)
                vid_data=[]
                for frame in frames_to_select:
                    try :
                        image=Image.open(os.path.join(vid_dir_path,frame))
                        image=image.resize((64, 64), Image.ANTIALIAS) 
                        datu=np.asarray(image)
                        normu_dat=datu/255
                        vid_data.append(normu_dat)
                    except :
                        vid_data.append(np.zeros_like(normu_dat)) # it is like padding as not all videos conatin 300 frames
                vid_data=np.array(vid_data)
                frames.append(vid_data)
        labels.append([d]*len(frames))
    return np.array(frames)

In [4]:
directory = 'UCF50'
frames = data_load_frames(directory)